# set up the environment

get miniconda with python 3.9

In [1]:
%%capture
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [2]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages')

set up the environment from StyleGAN 3 repo with conda + install/downgrade some packages that were causing errors

In [ ]:
!git clone -q https://github.com/NVlabs/stylegan3
!conda env update -q -n base -f stylegan3/environment.yml
!conda install -q -y psutil numpy==1.22.3 setuptools==58.0.4

connect with drive

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


get the data

In [ ]:
!cp -r 'drive/My Drive/anime.zip' .

# train the model from scratch

preprocess the data

In [ ]:
!python stylegan3/dataset_tool.py --source=anime.zip --dest=anime_64x64.zip \
    --transform=center-crop --resolution=64x64

100% 63565/63565 [05:37<00:00, 188.39it/s]


In [ ]:
DATA_PATH = 'anime_64x64.zip'
OUT_PATH = 'drive/My Drive/stylegan3_output'

activate tensorboard

In [ ]:
!conda install -q -y tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "$OUT_PATH"

train the model

In [ ]:
batch_size = 64
gamma = 1
snap = 10

In [ ]:
!python stylegan3/train.py --outdir="$OUT_PATH" --data="$DATA_PATH" \
    --cfg=stylegan3-t --cbase=16384 --workers=2 --gpus=1 --aug=noaug --mirror=1 \
    --batch=$batch_size --gamma=$gamma --snap=$snap

resume the training if interrupted

In [ ]:
resume_net = '00008-stylegan3-t-anime_resized-gpus1-batch64-gamma1/network-snapshot-000322.pkl'
resume_path = fr'{OUT_PATH}/{resume_net}'

In [ ]:
!python stylegan3/train.py --outdir="$OUT_PATH" --data="$DATA_PATH" \
    --cfg=stylegan3-t --cbase=16384 --workers=2 --gpus=1 --aug=noaug --mirror=1 \
    --batch=$batch_size --gamma=$gamma --snap=$snap --resume="$resume_path"

# transfer learning

In [ ]:
!python stylegan3/dataset_tool.py --source=anime.zip --dest=anime_256x256.zip \
    --transform=center-crop --resolution=256x256

100% 63565/63565 [05:37<00:00, 188.39it/s]


In [8]:
DATA_PATH = 'anime_256x256.zip'
OUT_PATH = 'drive/My Drive/stylegan3_output'

choose a model from the Nvidia repository and fine-tune it on our data with some layers freezed

In [ ]:
model = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-256x256.pkl'

In [ ]:
batch_size = 16
gamma = 1
snap = 2

In [ ]:
!python stylegan3/train.py --outdir="$OUT_PATH" --data="$DATA_PATH" \
    --cfg=stylegan3-t --cbase=16384 --workers=2 --gpus=1 --aug=noaug --metrics=none \
    --freezed=10 --batch=$batch_size --gamma=$gamma --snap=$snap --resume="$model"

# generate progress gif

In [5]:
!git clone -q https://github.com/kamilabielska/gans

In [6]:
import os
import glob
import matplotlib.pyplot as plt
import cv2

from gans.utils.vizualization import gif_from_image_folder

In [7]:
IMAGE_OUT_PATH = 'drive/My Drive/stylegan3_images'
PROGRESS_PATH = 'drive/My Drive/stylegan3_progress'

In [9]:
%%capture

for i, model_name in enumerate(glob.glob('**/network*.pkl', recursive=True, root_dir=OUT_PATH)):
    model_path = os.path.join(OUT_PATH, model_name)
    !python stylegan3/gen_images.py --outdir="$IMAGE_OUT_PATH" --seeds=42-45 --network="$model_path"

    fig, axes = plt.subplots(1, 4, figsize=(16,5))
    axes = axes.flatten()
    for j, filename in enumerate(os.listdir(IMAGE_OUT_PATH)):
        filepath = os.path.join(IMAGE_OUT_PATH, filename)
        image = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
        axes[j].imshow(image)
        axes[j].axis('off')
        os.remove(filepath)

    plt.savefig(os.path.join(PROGRESS_PATH, fr'{str(i)}.jpg'), bbox_inches='tight')
    plt.close(fig)

In [10]:
gif_from_image_folder(PROGRESS_PATH, 'drive/My Drive/stylegan3_progress.gif')

# generate interpolation video

In [19]:
final_model = fr'{OUT_PATH}/00004-stylegan3-t-anime_256x256-gpus1-batch16-gamma1/network-snapshot-000168.pkl'
video_path = 'drive/My Drive/stylegan3_inter.mp4'

In [ ]:
!python stylegan3/gen_video.py --output="$video_path" --trunc=1 --seeds=0-43 --grid=4x1 --network="$final_model"